In [ ]:
import socket, threading
import time
import mysql.connector

class Cliente_hilos(threading.Thread):
    #funcion para la instancia de hilos con los parametros
    def __init__(self, cliente_direction, cliente_socket):
        #creacion de hilo nuevo
        threading.Thread.__init__(self)
        self.csocket = cliente_socket
        #Mensaje de conexion cliente1 y hora
    def run(self):
        print("Conexion de cliente: ", cliente_direccion)
        print("A la hora ",time.strftime("%a %b %d %d %H:%M:%S %Y"))
        #Inicializacion de variable msg con cadena vacia
        mensaje = ''
        self.csocket.send("Bienvenido al servidor ".encode("UTF-8"))
        
        while True:
            #Obtencion de datos de cliente
            data = self.csocket.recv(2048)
            #pasamos los datos decodificados a la variable msg
            mensaje = data.decode()
            #Opcion para sincronizacion por Christian
            if mensaje=='1':
                print("Mensaje del cliente ",mensaje)
                #inicial=time.clock En desuso en la versión 3.8 de python
                inicial=time.process_time()
                inicio_tiempo=time.time()+inicial
                print(time.ctime(inicio_tiempo))
                print("tiempo vuelta",inicial)
                self.csocket.send(str(inicio_tiempo).encode("UTF-8"))
            #Opcion de sincronizacion por Barkley
            elif mensaje=='2':
                reloj_cliente=float(self.csocket.recv(2048))
                reloj_servidor=time.time()
                reloj_promedio=(reloj_cliente+reloj_servidor)/2
                self.csocket.send(str(reloj_promedio).encode("UTF-8"))
                print("El reloj promedio es ",time.ctime(reloj_promedio))
            #Opcion salida del servidor
            elif mensaje=='3':
                print("Cliente desconectando...")
                break
            #impresion del mensaje recibido por el cliente
            elif mensaje=='4':
                #Obtencion de datos de cliente
                data = self.csocket.recv(2048)
                #pasamos los datos decodificados a la variable msg
                mensaje = data.decode()
                
                #Base de datos mysql online creada con Clever Cloud
                mydb = mysql.connector.connect(host="bi91ppcu2sgz1cpwtou9-mysql.services.clever-cloud.com",
                     user="umwnowf1sh2nhwcj",
                     passwd="NrYK2cnE3AML56gDayCv",
                     db="bi91ppcu2sgz1cpwtou9")
                mycursor = mydb.cursor()
                
                #Verifica que el usuario exista en la base de datos
                sql = "SELECT COUNT(usuario) FROM CUENTA WHERE usuario='"+mensaje+"'"
                
                mycursor.execute(sql)

                myresult = mycursor.fetchone()
                self.csocket.send(str(myresult[0]).encode("UTF-8"))
                
                #Si existe el usuario se procede a esperar una accion por parte de el
                if myresult[0] == 1:
                    #En espera de una respuesta del cliente
                    usuario=mensaje
                    #Obtencion de datos de cliente
                    data = self.csocket.recv(2048)
                    #pasamos los datos decodificados a la variable msg
                    mensaje = data.decode()
                    #En caso de deposito
                    if mensaje=='1':
                        data = self.csocket.recv(2048)
                        #pasamos los datos decodificados a la variable msg
                        mensaje = data.decode()
                        
                        #Verifica el saldo del usuario en la base de datos
                        sql = "SELECT cantidad FROM CUENTA WHERE usuario='"+usuario+"'"
                        mycursor.execute(sql)

                        myresult = mycursor.fetchone()
                        #Calcula el saldo actualizado de la suma
                        deposito=myresult[0]+int(mensaje)
                        #Actualiza la base de datos
                        sql = "UPDATE CUENTA SET cantidad ="+str(deposito)+" WHERE usuario ='"+usuario+"'"
                        mycursor.execute(sql)
                        mydb.commit()
                        self.csocket.send(("HECHO").encode("UTF-8"))
                    #En caso de retiro
                    elif mensaje=='2':
                        data = self.csocket.recv(2048)
                        #pasamos los datos decodificados a la variable msg
                        mensaje = data.decode()
                        
                        #Verifica el saldo del usuario en la base de datos
                        sql = "SELECT cantidad FROM CUENTA WHERE usuario='"+usuario+"'"
                        mycursor.execute(sql)

                        myresult = mycursor.fetchone()
                        #Calcula el saldo actualizado de la resta
                        retiro=myresult[0]-int(mensaje)
                        #Actualiza la base de datos
                        sql = "UPDATE CUENTA SET cantidad ="+str(retiro)+" WHERE usuario ='"+usuario+"'"
                        mycursor.execute(sql)
                        mydb.commit()
                        self.csocket.send(("HECHO").encode("UTF-8"))
                    #En caso de consulta
                    elif mensaje=='3':
                        #Verifica el saldo del usuario en la base de datos
                        sql = "SELECT cantidad FROM CUENTA WHERE usuario='"+usuario+"'"
                        mycursor.execute(sql)

                        myresult = mycursor.fetchone()
                        self.csocket.send(str(myresult[0]).encode("UTF-8"))
            #En el caso de que aparezca un hilo vigilante
            elif mensaje=='Llega':
                #Regresa una respuesta ante el mensaje recibido por el vigilante\n",
                self.csocket.send('Llego'.encode("UTF-8"))
                print("Respondiendo a un hilo vigilante\n")
            else:
                print("Mensaje del cliente ",mensaje)
            
        #Mensaje al cliente desconectando
        print("Cliente ",cliente_direccion," disconnected...")

In [ ]:
#Inicializacion de variables
LOCALHOST="127.0.0.1" #ip de localhost
PORT=8080 #puerto a utilizar

server = socket.socket(socket.AF_INET,socket.SOCK_STREAM) #tipo de servidor a utilizar
server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) #Activacion de uso de servidor
server.bind((LOCALHOST, PORT)) #Creacion de servidor
#Mensajes de inicio de servidor
print("Servidor iniciado")
print("esperando peticiones de los clientes...")

while True:
    #Numero de clientes a escuchar
    server.listen(2)
    #Ontencion de socket y direccion del cliente
    cliente_sock, cliente_direccion = server.accept()
    #pasamos parametros a la funcion crear hilos
    nuevo_hilo=Cliente_hilos(cliente_direccion, cliente_sock)
    #iniciamos el hilo creado
    nuevo_hilo.start()